<a href="https://colab.research.google.com/github/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Đồ án Lập trình song song ứng dụng**

# Xoá background dựa trên ứng dụng segmentation của SLIC

## Giáo viên: Trần Trung Kiên

### Sinh viên:
- 1712568	KIM ĐÌNH LỘC	
- 1712787	NGUYỄN VĂN THÌN	
- 1712898	TRẦN VIỆT VĂN	


#1 MÔ TẢ ỨNG DỤNG

**Ứng dụng mà nhóm chọn là Xoá background dựa trên ứng dụng segmentation của SLIC**.  
- Input: Một tấm ảnh RGB.
  
  <img src="https://raw.githubusercontent.com/kimdinhloc/Applied_Parallel_Programming_HK2_2021_2022/main/dataset/Test10_15990.jpg" 
     width="640" 
     height="360" />

- Output: Ảnh đầu vào được xác định object, segmentation và remove background.  
<img src="https://raw.githubusercontent.com/kimdinhloc/dataset/main/Untitled.png" 
     width="165" 
     height="294" />


- Ý nghĩa của ứng dụng:
  - Hỗ trợ tách/lọc nền trong việc chỉnh sửa ảnh.
  - Là bước quan trọng trong các hệ thống sử dụng nhận diện vật thể.
  - Nghiên cứu về segmentation cũng như nghiên cứu về SLIC trong việc xoá nền ảnh.
- Lý do cần tăng tốc ứng dụng:
	- Ứng dụng sẽ chạy rất chậm nếu cài đặt tuần tự
	- Nhu cầu xử lý dữ liệu lớn, ảnh có độ phân giải cao hay việc xử lý theo thời gian thực
- Tiềm năng song song hóa của ứng dụng: 
  - Song song hoá việc chuyển từ hệ màu RGB sang hệ màu CELAB
  - Thực hiện đồng thời quá trình khởi tạo các cụm của SLIC.
  - Thực hiện đồng thời quá trình tính từ trung tâm cụm đến từng pixel trong vùng.
  - Thực hiện đồng thời quá trình tích chập trong edge detection.
- Thách thức:
  - Quá trình song song hoá việc kết nối các pixel trong SLIC có thể là một việc khó khăn.
  - Không thể song song hoá connected component labeling
  - Thời gian xử lý lâu hơn so với model Machine Learning/deep learning.
  - Độ chính xác không cao so với machine learning/deep learning.
  - Ít bài báo có chủ đề về việc lấy superpixels để thực hiện remove background hoặc tương tự.
  - Ít bài báo có chủ đề về việc lấy edge detections để hạn chế khả năng segmentation lỗi của superpixels

#2 CÀI ĐẶT TUẦN TỰ

##2.1 Thiết kế mô hình

<img src="https://raw.githubusercontent.com/kimdinhloc/dataset/main/PARALELL_PROGRAMING.jpg" 
     width="640" 
     height="360" />

## **HƯỚNG ĐI TUẦN TIẾP THEO**
- Tiếp cận phương pháp 
- Hoàn thiện code ở host
- Cố gắng có thể hoàn thành phần parallel của superpixels và objects detection.


## 2.2 Thử nghiệm

#3 CÀI ĐẶT SONG SONG

## 3.1 Mô hình song song hoá

## 3.2 Song Song hoá quá trình gom cụm của SLIC Superpixels

<img src="https://raw.githubusercontent.com/kimdinhloc/dataset/main/IMG/PARALELL_PROGRAMING/Slide2.JPG" 
     width="640" 
     height="360" />

## 3.2 Objects detection sử dụng bằng phương pháp ghép 2 phương pháp nhận diện biên cạnh.

<img src="https://raw.githubusercontent.com/kimdinhloc/dataset/main/IMG/PARALELL_PROGRAMING/Slide3.JPG" 
     width="640" 
     height="360" />

# TÀI LIỆU THAM KHẢO

Achanta, Radhakrishna, et al. Slic superpixels. No. REP_WORK. 2010. [Chi tiết tại đây!](https://infoscience.epfl.ch/record/149300)

Le, Cuong Vo, et al. "Superpixel-based background removal for accuracy salience person re-identification." 2016 IEEE International Conference on Consumer Electronics-Asia (ICCE-Asia). IEEE, 2016. [Chi tiết tại đây!](https://ieeexplore.ieee.org/abstract/document/7804806/)



Chaibou, Mahaman Sani, et al. "Adaptive strategy for superpixel-based region-growing image segmentation." Journal of Electronic Imaging 26.6 (2017): 061605. [Chi tiết tại đây!](https://arxiv.org/pdf/1803.06541.pdf)

John Canny. A computational approach to edge detection. Pattern Analysis and Machine Intelligence, IEEE Transactions on, (6):679–698, 1986. [Chi tiết tại đây!](https://canvas.stanford.edu/courses/98045/files/4183084/download?verifier=af00LZDbnDaaTurZAWvWhzOtJraLfk8DZsQqwuOy&wrap=1)